In [1]:
# Step 1: Install Libraries
!pip install tensorflow numpy matplotlib

In [2]:
# Step 2: Import Libraries
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import files

In [3]:
# Step 3: Upload Input Image
uploaded = files.upload()
input_image_path = list(uploaded.keys())[0]

Saving mahfuj.jpg to mahfuj.jpg


In [10]:
# Step 4: Load and Preprocess Input Image
def load_and_preprocess_image(image_path, target_size=(32, 32)): # Changed target_size to (32, 32)
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = (img - 127.5) / 127.5  # Normalize to [-1, 1]
    return img

input_image = load_and_preprocess_image(input_image_path)
input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension

In [11]:
# Step 5: Define Generator and Discriminator
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((8, 8, 256)))
    assert model.output_shape == (None, 8, 8, 256)

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 8, 8, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 16, 16, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 32, 32, 3)

    return model

def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[32, 32, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [12]:
# Step 6: Define Loss Functions and Optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [13]:
# Step 7: Define Training Loop
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [14]:
# Step 8: Train the GAN
EPOCHS = 50
BATCH_SIZE = 1

generator = make_generator_model()
discriminator = make_discriminator_model()

for epoch in range(EPOCHS):
    start = time.time()
    train_step(input_image)
    print(f'Epoch {epoch + 1}, Time: {time.time() - start}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1, Time: 3.865213394165039
Epoch 2, Time: 0.04369306564331055
Epoch 3, Time: 0.043544769287109375
Epoch 4, Time: 0.03972363471984863
Epoch 5, Time: 0.037901878356933594
Epoch 6, Time: 0.03796696662902832
Epoch 7, Time: 0.038489341735839844
Epoch 8, Time: 0.038201332092285156
Epoch 9, Time: 0.03982210159301758
Epoch 10, Time: 0.04320645332336426
Epoch 11, Time: 0.05109262466430664
Epoch 12, Time: 0.0379025936126709
Epoch 13, Time: 0.0396883487701416
Epoch 14, Time: 0.04013419151306152
Epoch 15, Time: 0.03984522819519043
Epoch 16, Time: 0.04354143142700195
Epoch 17, Time: 0.044820547103881836
Epoch 18, Time: 0.03816556930541992
Epoch 19, Time: 0.04218006134033203
Epoch 20, Time: 0.03812551498413086
Epoch 21, Time: 0.04120779037475586
Epoch 22, Time: 0.038841962814331055
Epoch 23, Time: 0.04061484336853027
Epoch 24, Time: 0.03828167915344238
Epoch 25, Time: 0.037938833236694336
Epoch 26, Time: 0.03837108612060547
Epoch 27, Time: 0.039861202239990234
Epoch 28, Time: 0.040199995040893

In [15]:
# Step 9: Generate and Save Output Image
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    plt.figure(figsize=(4, 4))
    plt.imshow(predictions[0, :, :, :] * 0.5 + 0.5)  # Rescale to [0, 1]
    plt.axis('off')
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

noise = tf.random.normal([1, 100])
generate_and_save_images(generator, EPOCHS, noise)